In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce
import re

In [44]:
def find_subject(subject_code='A001',path='LOG/'):
    """
    Read path, then search in given path person with
    given subject code'
    """
    files = [file for file in os.listdir('LOG/') if 'log' in file]
    return [file for file in files if subject_code in file]

def read_file(filename):
    """
    Read file, check whether it contains proper
    columns (based on first one) and return 
    DataFrame.
    """
    #read file, skiping rows with metadata
    file = pd.read_csv(filename,sep="\t",skiprows=2)
    #check columns (whether first one is subject)
    if np.any(file.columns.str.contains('Subject')):
        #delete Response (irrelevant for this study, but 
        #ther are some troubles with it)
        file = file.loc[file['Event Type'] != 'Response']
        #if columns are proper, restart index and return
        return file.reset_index()
    else:
        #if columns are not proper, read with skipping
        #different n of rows
        file = pd.read_csv(filename,"\t",skiprows=3)
        #drop unnecessary rows
        file = file.dropna(subset = ['Subject'])
        #delete Response (irrelevant for this study, but 
        #ther are some troubles with it)
        file = file.loc[file['Event Type'] != 'Response']
        #if columns are proper, restart index and return
        return file.reset_index()
    
def get_arg_block(data,blockname='block'):
    """Get arg of blocks in data"""
    return (data
            .loc[(data['Code']
                  .str.lower()
                  .str.contains(blockname.lower())),:]
            .index)

def time_correction(data):
    """
    Corrects time in data from Presentation. 
    Reduce all Time column by time value of first event, 
    then divides by 10000 (to turn into ms).
    Returns whole DataFrame.
    """
    data['Time'] = (data['Time'] - data['Time'].iloc[0])/10000
    return data

def create_file(filename,fixed_window=0):
    """
    Creates file with given filename
    """
    df = read_file(filename)
    df = time_correction(df)
    args = get_arg_block(df,r'S*P')
    df = df.loc[args]

    result = pd.DataFrame(columns=['Code','Onset','Offset'])
    for ind in range(len(df)):
        part = df.iloc[ind][['Code','Time']]
        try:
            if 'inter' in part['Code']:
                offset = part['Time'] + 15
                duration = 15
            else:
                offset = df.iloc[ind+1]['Time']
                duration = 13
            if fixed_duration>0:
                duration = fixed_duration
        except:
            pass

        result = pd.concat([result,
                            pd.DataFrame({'Code':part['Code'],
                             'Onset':part['Time'],
                             'Offset':offset,
                             'Duration':duration},index=[0])])
    return result.reset_index()


def create_file_end_recording(filename):
    """
    Creates file with given filename; 
    it is alternative way, in which 
    """
    df = read_file(filename)
    df = time_correction(df)
    args = get_arg_block(df,r'S*P')
    df = df.loc[args]

    result = pd.DataFrame(columns=['Code','Onset','Offset'])
    for ind in range(len(df)):
        part = df.iloc[ind][['Code','Time']]
        try:
            if 'end' in part['Code']:
                offset = part['Time'] + 15
                duration = 15
            else:
                offset = df.iloc[ind+1]['Time']
                duration = 13
        except:
            pass

        result = pd.concat([result,pd.DataFrame({'Code':part['Code'],
                      'Onset':part['Time'],
                      'Offset':offset,
                      'Duration':duration},index=[0])])
    return result.reset_index()

def generate_timing_conditions(data,prefix):
    for condition,name in zip(['S','SN'],
                              ['krytyka','neutralne']):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'{condition}\d_P[{part}]{modality_sign}'),]
                 .to_csv(f'{prefix}_{name}_{modality}_P{part}.tsv'))
    if any(data['Code'].str.contains(fr'S\d_P\d_T')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}_therapy.tsv'))
    elif any(data['Code'].str.contains(fr'SN\d_P\d_K')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['Code']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}.tsv'))
                
def FSL_fix(data):
    """
    Return data, with columns necessary for analysis in FSL.
    """
    data['weight'] = np.ones(len(data))
    data.columns = ['index','trial_type','onset','offset','duration','weight']
    data['onset'] = np.round(data['onset'])
    return data.loc[:,['onset','duration','weight','trial_type']]

def generate_timing_conditions_FSL(data,prefix):
    data = FSL_fix(data)
    for condition,name in zip(['S','SN'],
                              ['krytyka','neutralne']):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'{condition}\d_P[{part}]{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_{name}_{modality}_P{part}.tsv','\t'))
    if any(data['trial_type'].str.contains(fr'S\d_P\d_T')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'S\d_P[{part}]_T{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}_therapy.tsv','\t'))
    elif any(data['trial_type'].str.contains(fr'SN\d_P\d_K')):
        for modality_sign,modality in zip(['$','inter'],
                                          ['s','i']):
            for part in ['1','2','3','123','4','5','6','7','4567']:
                (data.loc[data['trial_type']
                      .str
                      .contains(fr'SN\d_P[{part}]_K{modality_sign}'),['onset','duration','weight']]
                 .to_csv(f'{prefix}_therapy_{modality}_P{part}.tsv','\t'))

In [49]:
file = read_file('LOG/JM2022_B004_TP1-Run2_functionalScan.log')
file = create_file('LOG/JM2022_B004_TP1-Run2_functionalScan.log')

# generate_timing_conditions_FSL(file,'test')
# file['weight'] = np.ones(len(file))
# # uno = file.loc[:,['onset','duration','weight','trial_type']]
# # uno.columns = ['onset','duration','weight','trial_type']
# # uno

# FSL_fix(file)
file
generate_timing_conditions_FSL(file,'test')

In [48]:
path='LOG/'
for raw_file in os.listdir('LOG/'):
    #check whether file contains subject's code
    if len(re.findall(r'b\d+',raw_file.lower()))>0:
        #if it contains code, try create directory with subject code,
        #if this directory exists, pass this action.
        subject_code = re.findall(r'b\d+',raw_file.lower())[0].upper()
        try:
            os.mkdir(subject_code)
            os.mkdir(f'{subject_code}\ses-1')
            os.mkdir(f'{subject_code}\ses-2')
        except FileExistsError:
            pass
        #
        files = find_subject(subject_code,path)
        for filename in files:
            print(filename)
            try:
                result = create_file(f"{path}{filename}")
                run = re.findall(r'Run\d',filename)[0]
                if 'tp1' in filename.lower():
                    generate_timing_conditions_FSL(result,f"{subject_code}\ses-1\\{subject_code}{run}")
                elif 'tp5' in filename.lower():
                    generate_timing_conditions_FSL(result,f"{subject_code}\ses-2\\{subject_code}{run}")
            except Exception as e:
                print(f"Problem with {filename}\n{e}")

B031_neeutralny_test-Baseline_T1.log
Problem with B031_neeutralny_test-Baseline_T1.log
single positional indexer is out-of-bounds
B031_T17_test-Baseline_T1.log
Problem with B031_T17_test-Baseline_T1.log
single positional indexer is out-of-bounds
B031_test-Run1_functionalScan.log
Problem with B031_test-Run1_functionalScan.log
single positional indexer is out-of-bounds
B031_test-Run2_functionalScan.log
B031_test-Run2_functionalScan1.log
Problem with B031_test-Run2_functionalScan1.log
single positional indexer is out-of-bounds
B031_Test1-Baseline_T1.log
Problem with B031_Test1-Baseline_T1.log
Cannot mask with non-boolean array containing NA / NaN values
B031_Test1-Run1_functionalScan.log
B031_Test1-TestGlosnosci_FieldMap.log
Problem with B031_Test1-TestGlosnosci_FieldMap.log
list index out of range
JM22a_B031_TP1-Run1_functionalScan.log
Problem with JM22a_B031_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B031_TP1-Run1_functionalScan1.log
JM22a_B031_TP1-Run2

JM22a_B031_TP1-Run2_functionalScan.log
JM22a_B031_TP5-Run1_functionalScan.log
JM22a_B031_TP5-Run2_functionalScan.log
B046_T02_tp1-Run1_functionalScan.log
B046_T02_tp1-Run2_functionalScan.log
JM2022_B046_TP5-Run1_functionalScan.log
JM2022_B046_TP5-Run2_functionalScan.log
B046_T02_tp1-Run1_functionalScan.log
B046_T02_tp1-Run2_functionalScan.log
JM2022_B046_TP5-Run1_functionalScan.log
JM2022_B046_TP5-Run2_functionalScan.log
B251_T06_TP1-Run1_functionalScan.log
B251_T06_TP1-Run2_functionalScan.log
JM2022_B251_TP5-Run1_functionalScan.log
JM2022_B251_TP5-Run2_functionalScan.log
JM22a_B251_TP5-Run1_functionalScan.log
Problem with JM22a_B251_TP5-Run1_functionalScan.log
Can only use .str accessor with string values!
B251_T06_TP1-Run1_functionalScan.log
B251_T06_TP1-Run2_functionalScan.log
JM2022_B251_TP5-Run1_functionalScan.log
JM2022_B251_TP5-Run2_functionalScan.log
JM22a_B251_TP5-Run1_functionalScan.log
Problem with JM22a_B251_TP5-Run1_functionalScan.log
Can only use .str accessor with string

JM22a_B254_TP1-Run1_functionalScan.log
JM22a_B254_TP1-Run2_functionalScan.log
JM2022_B256_TP1-Run1_functionalScan.log
Problem with JM2022_B256_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM2022_B256_TP1-Run1_functionalScan1.log
JM2022_B256_TP1-Run2_functionalScan.log
Problem with JM2022_B256_TP1-Run2_functionalScan.log
single positional indexer is out-of-bounds
JM2022_B256_TP1-Run2_functionalScan1.log
JM2022_B256_TP1-Run1_functionalScan.log
Problem with JM2022_B256_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM2022_B256_TP1-Run1_functionalScan1.log
JM2022_B256_TP1-Run2_functionalScan.log
Problem with JM2022_B256_TP1-Run2_functionalScan.log
single positional indexer is out-of-bounds
JM2022_B256_TP1-Run2_functionalScan1.log
JM2022_B256_TP1-Run1_functionalScan.log
Problem with JM2022_B256_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM2022_B256_TP1-Run1_functionalScan1.log
JM2022_B256_TP1-Run2_functionalScan.l

JM22a_B031_TP5-Run1_functionalScan.log
JM22a_B031_TP5-Run2_functionalScan.log
JM22a_B045_TP5-Run1_functionalScan.log
JM22a_B045_TP5-Run2_functionalScan.log
JM22_B045_TP1-Run1_functionalScan.log
JM22_B045_TP1-Run2_functionalScan.log
JM22a_B045_TP5-Run1_functionalScan.log
JM22a_B045_TP5-Run2_functionalScan.log
JM22_B045_TP1-Run1_functionalScan.log
JM22_B045_TP1-Run2_functionalScan.log
JM2022_B052_TP1-Run1_functionalScan.log
JM2022_B052_TP1-Run2_functionalScan.log
JM22a_B052_TP5-Run1_functionalScan.log
Problem with JM22a_B052_TP5-Run1_functionalScan.log
Can only use .str accessor with string values!
JM22a_B052_TP5-Run1_functionalScan1.log
JM22a_B052_TP5-Run2_functionalScan.log
JM2022_B052_TP1-Run1_functionalScan.log
JM2022_B052_TP1-Run2_functionalScan.log
JM22a_B052_TP5-Run1_functionalScan.log
Problem with JM22a_B052_TP5-Run1_functionalScan.log
Can only use .str accessor with string values!
JM22a_B052_TP5-Run1_functionalScan1.log
JM22a_B052_TP5-Run2_functionalScan.log
JM2022_B052_TP1-Run1

JM22a_B226_TP1-Run1_functionalScan.log
JM22a_B226_TP1-Run2_functionalScan.log
JM22a_B226_TP5-Run1_functionalScan.log
Problem with JM22a_B226_TP5-Run1_functionalScan.log
Can only use .str accessor with string values!
JM22a_B226_TP5-Run1_functionalScan1.log
Problem with JM22a_B226_TP5-Run1_functionalScan1.log
Can only use .str accessor with string values!
JM22a_B226_TP5-Run1_functionalScan2.log
Problem with JM22a_B226_TP5-Run1_functionalScan2.log
Cannot mask with non-boolean array containing NA / NaN values
JM22a_B226_TP5-Run1_functionalScan3.log
JM22a_B226_TP5-Run2_functionalScan.log
JM22a_B226_TP1-Run1_functionalScan.log
JM22a_B226_TP1-Run2_functionalScan.log
JM22a_B226_TP5-Run1_functionalScan.log
Problem with JM22a_B226_TP5-Run1_functionalScan.log
Can only use .str accessor with string values!
JM22a_B226_TP5-Run1_functionalScan1.log
Problem with JM22a_B226_TP5-Run1_functionalScan1.log
Can only use .str accessor with string values!
JM22a_B226_TP5-Run1_functionalScan2.log
Problem with J

...

JM22a_B362_TP5-Run1_functionalScan.log
JM22a_B362_TP5-Run2_functionalScan.log
JM22a_B362_TP1-Run1_functionalScan.log
Problem with JM22a_B362_TP1-Run1_functionalScan.log
Cannot mask with non-boolean array containing NA / NaN values
JM22a_B362_TP1-Run1_functionalScan1.log
JM22a_B362_TP1-Run2_functionalScan.log
JM22a_B362_TP5-Run1_functionalScan.log
JM22a_B362_TP5-Run2_functionalScan.log
JM22a_B366_TP1-Run1_functionalScan.log
Problem with JM22a_B366_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B366_TP1-Run1_functionalScan1.log
Problem with JM22a_B366_TP1-Run1_functionalScan1.log
Cannot mask with non-boolean array containing NA / NaN values
JM22a_B366_TP1-Run1_functionalScan2.log
JM22a_B366_TP5-Run1_functionalScan.log
JM22a_B366_TP5-Run2_functionalScan.log
JM22_B366_TP1-Run2_functionalScan.log
JM22a_B366_TP1-Run1_functionalScan.log
Problem with JM22a_B366_TP1-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B366_TP1-Run1_functionalSca

JM22a_B413_TP5-Run1_functionalScan.log
Problem with JM22a_B413_TP5-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B413_TP5-Run1_functionalScan1.log
Problem with JM22a_B413_TP5-Run1_functionalScan1.log
single positional indexer is out-of-bounds
JM22a_B413_TP5-Run1_functionalScan2.log
Problem with JM22a_B413_TP5-Run1_functionalScan2.log
single positional indexer is out-of-bounds
JM22a_B413_TP5-Run1_functionalScan3.log
Problem with JM22a_B413_TP5-Run1_functionalScan3.log
single positional indexer is out-of-bounds
JM22a_B413_TP5-Run2_functionalScan.log
Problem with JM22a_B413_TP5-Run2_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B413_TP1-Run1_functionalScan.log
JM22a_B413_TP1-Run2_functionalScan.log
JM22a_B413_TP5-Run1_functionalScan.log
Problem with JM22a_B413_TP5-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B413_TP5-Run1_functionalScan1.log
Problem with JM22a_B413_TP5-Run1_functionalScan1.log
single positional i

JM22a_B448_TP1-Run1_functionalScan.log
JM22a_B448_TP1-Run2_functionalScan.log
JM22a_B448_TP5-Run1_functionalScan.log
JM22a_B448_TP5-Run2_functionalScan.log
JM22a_B448_TP1-Run1_functionalScan.log
JM22a_B448_TP1-Run2_functionalScan.log
JM22a_B448_TP5-Run1_functionalScan.log
JM22a_B448_TP5-Run2_functionalScan.log
JM22a_B448_TP1-Run1_functionalScan.log
JM22a_B448_TP1-Run2_functionalScan.log
JM22a_B448_TP5-Run1_functionalScan.log
JM22a_B448_TP5-Run2_functionalScan.log
JM22a_B448_TP1-Run1_functionalScan.log
JM22a_B448_TP1-Run2_functionalScan.log
JM22a_B448_TP5-Run1_functionalScan.log
JM22a_B448_TP5-Run2_functionalScan.log
JM22a_B452_TP1-Run1_functionalScan.log
JM22a_B452_TP1-Run2_functionalScan.log
JM22a_B452_TP5-Run1_functionalScan.log
Problem with JM22a_B452_TP5-Run1_functionalScan.log
single positional indexer is out-of-bounds
JM22a_B452_TP5-Run2_functionalScan.log
JM22a_B452_TP1-Run1_functionalScan.log
JM22a_B452_TP1-Run2_functionalScan.log
JM22a_B452_TP5-Run1_functionalScan.log
Problem 

JM22A_B541_TP5-Run1_functionalScan.log
JM22A_B541_TP5-Run2_functionalScan.log
JM22a_B541_TP1-Run1_functionalScan.log
JM22a_B541_TP1-Run2_functionalScan.log
JM22A_B541_TP5-Run1_functionalScan.log
JM22A_B541_TP5-Run2_functionalScan.log
JM22a_B541_TP1-Run1_functionalScan.log
JM22a_B541_TP1-Run2_functionalScan.log
JM22A_B541_TP5-Run1_functionalScan.log
JM22A_B541_TP5-Run2_functionalScan.log
JM22a_B541_TP1-Run1_functionalScan.log
JM22a_B541_TP1-Run2_functionalScan.log
JM22A_B541_TP5-Run1_functionalScan.log
JM22A_B541_TP5-Run2_functionalScan.log
JM22a_B545_TP1-Run1_functionalScan.log
JM22a_B545_TP1-Run2_functionalScan.log
JM22a_B545_TP5-Run1_functionalScan.log
JM22a_B545_TP5-Run2_functionalScan.log
JM22a_B545_TP1-Run1_functionalScan.log
JM22a_B545_TP1-Run2_functionalScan.log
JM22a_B545_TP5-Run1_functionalScan.log
JM22a_B545_TP5-Run2_functionalScan.log
JM22a_B545_TP1-Run1_functionalScan.log
JM22a_B545_TP1-Run2_functionalScan.log
JM22a_B545_TP5-Run1_functionalScan.log
JM22a_B545_TP5-Run2_funct

JM22a_B676_TP5-Run1_functionalScan.log
JM22a_B676_TP5-Run2_functionalScan.log
B676_T14-Run1_functionalScan.log
Problem with B676_T14-Run1_functionalScan.log
Cannot mask with non-boolean array containing NA / NaN values
jm22a_B676_TP1-Run1_functionalScan.log
jm22a_B676_TP1-Run2_functionalScan.log
JM22a_B676_TP5-Run1_functionalScan.log
JM22a_B676_TP5-Run2_functionalScan.log
B676_T14-Run1_functionalScan.log
Problem with B676_T14-Run1_functionalScan.log
Cannot mask with non-boolean array containing NA / NaN values
jm22a_B676_TP1-Run1_functionalScan.log
jm22a_B676_TP1-Run2_functionalScan.log
JM22a_B676_TP5-Run1_functionalScan.log
JM22a_B676_TP5-Run2_functionalScan.log
B676_T14-Run1_functionalScan.log
Problem with B676_T14-Run1_functionalScan.log
Cannot mask with non-boolean array containing NA / NaN values
jm22a_B676_TP1-Run1_functionalScan.log
jm22a_B676_TP1-Run2_functionalScan.log
JM22a_B676_TP5-Run1_functionalScan.log
JM22a_B676_TP5-Run2_functionalScan.log
JM22a_B687_TP1-Run1_functionalS

In [38]:
result
condition='S'
part=7
modality_sign='_Tinter'
(result.loc[result['trial_type'].str.contains(fr'{condition}\d_P[{part}]{modality_sign}')])
# if any(result['trial_type'].str.contains(fr'S\d_P\d_T')):
#     print(True)

,index,Code,Onset,Offset,Duration
53,0,S1_P7_Tinter,1009.2155,1024.2155,15.0


In [19]:
result
files = find_subject(person_code,path)


['Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run1',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run1',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run2',
 'Run1',
 'Run1',
 'Run1',
 'Run2',
 'Run2',
 

In [45]:
filename

'B031_neeutralny_test-Baseline_T1.log'

In [17]:
files

['B046_T02_tp1-Run1_functionalScan.log',
 'B046_T02_tp1-Run2_functionalScan.log',
 'JM2022_B046_TP5-Run1_functionalScan.log',
 'JM2022_B046_TP5-Run2_functionalScan.log']